## LangChain Expression Language(LCEL)

https://python.langchain.com/v0.1/docs/expression_language/why/

### 기본 구조: 프롬프트 + 모델 + 출력 파서


In [4]:
pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


True

In [7]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

os.environ['LANGCHAIN_PROJECT'] = 'LCEL'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[LANGCHAIN_PROJECT]
LCEL


In [13]:
pip install langchain_teddynote

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_teddynote-0.3.6-py3-none-any.whl (38 kB)
  Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)
  Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)
  Using cached olefile-0.47-py2.py3-none-any.whl (114 kB)
  Using cached langchain-0.3.3-py3-none-any.whl (1.0 MB)
  Using cached openai-1.51.2-py3-none-any.whl (383 kB)
  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
  Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)
  Using cached deepl-1.19.1-py3-none-any.whl (35 kB)
  Using cached langgraph-0.2.35-py3-none-any.whl (108 kB)
  Using cached kiwipiepy-0.19.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.5 MB)
  Using cached pinecone_text-0.9.0-py3-none-any.whl (23 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Preparing metada

In [14]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL


### 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿
  - `template`: 템플릿 문자열. 문자열 내에서 중괄호 `{}`는 변수를 나타냄
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의함

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트

In [15]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

In [16]:
pip install langchain langchain-openai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [17]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [18]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [19]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [20]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0.1,
)

### Chain 생성

#### LCEL(LangChain Expression Language)


```
chain = prompt | model | output_parser
```

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달


In [21]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fcf13b7eec0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fcf13b91000>, root_client=<openai.OpenAI object at 0x7fcf13b7cfa0>, root_async_client=<openai.AsyncOpenAI object at 0x7fcf13b7ef20>, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태(키: 값)로 입력값을 전달
- invoke() 함수 호출 시, 입력값을 전달

In [22]:
# input 딕셔너리에 주제 설정
input = {"topic": "인공지능의 학습 방법"}

In [23]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달
# 이를 통해 AI 모델이 생성한 메시지를 반환
chain.invoke(input)

AIMessage(content='인공지능의 학습 방법은 크게 지도 학습, 비지도 학습, 강화 학습으로 나눌 수 있습니다.\n\n1. 지도 학습: 이는 가장 일반적인 학습 방법으로, 입력 데이터와 정답 데이터를 모두 제공하여 모델을 학습시키는 방법입니다. 예를 들어, 고양이 사진과 해당 고양이의 라벨(고양이)을 함께 제공하여 모델이 이미지를 분류할 수 있도록 학습시키는 것이 지도 학습의 한 예입니다.\n\n2. 비지도 학습: 이는 정답 데이터 없이 입력 데이터만을 이용하여 모델을 학습시키는 방법입니다. 모델은 데이터의 패턴이나 구조를 스스로 학습하여 클러스터링, 차원 축소, 이상 탐지 등의 작업을 수행할 수 있습니다.\n\n3. 강화 학습: 이는 보상을 통해 모델을 학습시키는 방법으로, 모델이 특정 행동을 취했을 때 보상을 받거나 처벌을 받으면서 학습합니다. 예를 들어, 게임에서 승리하면 보상을 받고 패배하면 처벌을 받는 방식으로 모델을 학습시키는 것이 강화 학습의 한 예입니다.\n\n이러한 다양한 학습 방법을 조합하여 인공지능 모델을 효과적으로 학습시킬 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 436, 'prompt_tokens': 30, 'total_tokens': 466, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-652e435a-f4e2-4278-9c55-477dc23e51ed-0', usage

In [24]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능의 학습 방법은 주로 머신러닝과 딥러닝 기술을 사용합니다. 이들은 데이터를 활용하여 컴퓨터가 스스로 패턴을 학습하도록 하는 방법입니다. 

1. 머신러닝: 머신러닝은 데이터를 바탕으로 컴퓨터가 패턴을 학습하는 방법입니다. 이를 위해 데이터를 입력하고 모델을 훈련시켜 원하는 결과를 출력하도록 합니다. 이때 모델은 데이터의 패턴을 파악하여 예측이나 분류를 수행합니다.

2. 딥러닝: 딥러닝은 인공신경망을 사용하여 복잡한 문제를 해결하는 방법입니다. 여러 층의 인공신경망을 통해 데이터의 다양한 특징을 학습하고 판단합니다. 이를 통해 컴퓨터는 사람과 유사한 학습 능력을 가질 수 있습니다.

이러한 방법을 통해 인공지능은 데이터를 통해 스스로 학습하고 발전하며, 다양한 분야에서 인간을 도와주는 기술로 활용되고 있습니다.

### 출력파서(Output Parser)


In [25]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [26]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fcf13b7eec0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fcf13b91000>, root_client=<openai.OpenAI object at 0x7fcf13b7cfa0>, root_async_client=<openai.AsyncOpenAI object at 0x7fcf13b7ef20>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [27]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능의 학습 원리"}
chain.invoke(input)

'인공지능은 데이터를 입력으로 받아서 패턴을 학습하고, 이를 바탕으로 새로운 데이터를 분석하거나 결정을 내리는 시스템입니다. 이를 위해서 인공지능은 먼저 학습 데이터를 활용하여 모델을 만들고, 이 모델을 통해 데이터의 패턴을 찾아내는 과정을 거칩니다. 이렇게 찾아낸 패턴을 기반으로 인공지능은 새로운 데이터를 분석하고 예측합니다. 이러한 과정을 통해 인공지능은 스스로 데이터를 이해하고 판단할 수 있게 됩니다.'

In [28]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능의 학습 원리는 주로 기계학습(머신 러닝)과 딥러닝(신경망) 기술을 이용합니다. 이러한 기술은 데이터를 분석하고 패턴을 학습하여 문제를 해결하는 방식으로 작동합니다.

기계학습은 주어진 데이터를 분석하여 패턴을 찾아내고, 이를 바탕으로 모델을 학습시킵니다. 모델은 입력 데이터를 받아들이고, 이를 처리하여 원하는 결과를 출력합니다. 이 과정에서 모델은 오차를 최소화하는 방향으로 학습을 진행하며, 점차적으로 정확도를 향상시킵니다.

딥러닝은 인공신경망을 사용하여 복잡한 문제를 해결하는 기술로, 기계학습의 한 분야입니다. 인공신경망은 뇌의 구조를 모방하여 설계된 모델로, 입력층, 은닉층, 출력층으로 구성되어 있습니다. 각 층은 노드로 연결되어 있고, 각 노드는 가중치와 활성화 함수를 가지고 있습니다. 학습 과정에서 이러한 가중치가 조정되어 최적의 결과를 찾아냅니다.

결론적으로, 인공지능의 학습 원리는 데이터를 분석하고 패턴을 학습하여 문제를 해결하는 과정으로 요약할 수 있습니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 가능
- `model_name` 역시 변경하여 테스트가 가능

In [29]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

In [30]:
# 체인을 구성
chain = prompt | model | output_parser

In [31]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
  **Waiter:** Good evening! Welcome to our restaurant. How many people are in your party?  
  **You:** Good evening! It's just me.  
  **Waiter:** Great! Here’s the menu. Can I get you something to drink while you look it over?  
  **You:** Yes, I’d like a glass of water, please.  
  **Waiter:** Sure! Are you ready to order, or do you need more time?  
  **You:** I think I'm ready. I would like the grilled chicken with a side of vegetables.  
  **Waiter:** Excellent choice! Would you like anything else?  
  **You:** No, that will be all for now. Thank you!  
  **Waiter:** You're welcome! I’ll be right back with your order.

- 한글 해석:  
  **웨이터:** 좋은 저녁입니다! 저희 식당에 오신 것을 환영합니다. 몇 분이신가요?  
  **당신:** 좋은 저녁입니다! 저 혼자입니다.  
  **웨이터:** 좋습니다! 여기 메뉴입니다. 메뉴를 보시면서 음료수는 무엇을 드릴까요?  
  **당신:** 네, 물 한 잔 주세요.  
  **웨이터:** 알겠습니다! 주문할 준비가 되셨나요, 아니면 더 필요하신가요?  
  **당신:** 저는 준비가 된 것 같아요. 그릴에 구운 치킨과 채소 사이드를 주문할게요.  
  **웨이터:** 훌륭한 선택입니다! 다른 것 더 필요하신가요?  
  **당신:** 아니요, 지금은 그걸로 충분해요. 감사합니다!  
  **웨

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

### [실습] 템플릿을 변경하여 나만의 여행 가이드 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 여행 가이드
2. 3일간 가성비 여행 계획을 세워주는 챗봇 생성
3. `{question}` 에는 여행갈 나라와 도시를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [36]:
tour_guide_template = """
당신은 여행지의 가이드를 담당하는 10년차 여행가이드입니다. 임의의 여행지에 대한 가성비 높은 3일치 계획을 세워주세요
양식은 [Format]을 참고하여 작성해주세요

# 상황:
{question}

#Format:
- 사용자의 질문:
- 여행 계획:
    -1일차:
    -2일차:
    -3일차:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(tour_guide_template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name='gpt-4o-mini')

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

# 체인을 구성
chain = prompt | model | output_parser

In [37]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question:영국 런던을 여행하고 싶은데 계획을 세워줘."}))
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "대한민국 부산을 여행하고 싶으니 계획을 세워줘."})
# 스트리밍 출력
stream_response(answer)

- 사용자의 질문: 영국 런던을 여행하고 싶은데 계획을 세워줘.
- 여행 계획:
    - 1일차:
        - 아침: 브릭 레인 마켓에서 로컬 푸드로 아침 식사 (예: 베이글과 커피)
        - 오전: 타워 브리지와 런던 타워 방문 (입장료 할인 가능 여부 확인)
        - 점심: 타워 브리지 근처의 펍에서 전통 영국식 점심 (예: 피시 앤 칩스)
        - 오후: 세인트 폴 대성당 외부 관람 후, 밀레니엄 브릿지를 따라 테이트 모던 미술관까지 산책
        - 저녁: 리버사이드의 레스토랑에서 저녁 식사 (예: 버거 또는 파스타)
        - 야경: 런던 아이에서 야경 감상 (예약 시 할인 가능)

    - 2일차:
        - 아침: 호텔 근처 카페에서 간단한 아침 식사
        - 오전: 버킹엄 궁전에서 근위병 교대식 관람 (무료)
        - 점심: 세인트 제임스 파크에서 피크닉 (미리 샌드위치와 음료 구매)
        - 오후: 내셔널 갤러리 방문 (입장 무료) 후 코OVENT GARDEN 탐방
        - 저녁: 코벤트 가든의 푸드 마켓에서 다양한 스트리트 푸드 시식
        - 공연: 웨스트 엔드에서 뮤지컬 관람 (미리 예약 시 할인 가능)

    - 3일차:
        - 아침: 호텔 조식 또는 근처 베이커리에서 패스트리와 커피
        - 오전: 대영 박물관 방문 (입장 무료, 기부 가능)
        - 점심: 박물관 근처의 현지 레스토랑에서 런치 스페셜
        - 오후: 캠든 마켓 탐방 및 쇼핑 (개성 있는 상점들 방문)
        - 저녁: 로컬 레스토랑에서 마지막 저녁 식사 (추천: 인도 혹은 이탈리안)
        - 야경: 템즈강 유람선 타기 (저녁 시간대에 경치 감상)
- 사용자의 질문: 부산을 여행하고 싶어요. 가성비 높은 3일치 계획을 세워주세요.
- 여행 계획:
    - 1일차:
        - 오전: 해운대 해수욕장 산책 및 조

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청

# 스트리밍 출력


### [실습] 템플릿을 변경하여 나만의 요리사 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 셰프
2. 냉장고 속 재료(여러 재료도 가능)를 입력으로 받아 요리명과 레시피 출력
3. `{food}` 에는 여행갈 냉장고 속 재료를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [42]:
# 코드 작성
my_cheif = """
당신은 숙련도 높은 10년차 셰프입니다. 냉장고 속 재료를 입력받아 그 재료를 활용한 요리명과 레시피를 출력해주세요.
양식은 [format]을 참조하세요.

#상황:
{question}

#format:
- 요리명:
- 레시피:
"""

prompt = PromptTemplate.from_template(my_cheif)

model = ChatOpenAI(model_name='gpt-4o-mini')

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [43]:
answer = chain.stream({'question' :'파스타면, 삼겹살, 올리브유, 다진마늘, 청양고추, 버섯플레이크, 후추, 소금, 파슬리가루, 슬라이스치즈'})

stream_response(answer)

- 요리명: 매콤한 삼겹살 파스타

- 레시피:
  1. **재료 준비**: 파스타면 200g, 삼겹살 150g, 올리브유 2큰술, 다진마늘 1큰술, 청양고추 2개, 버섯플레이크 2큰술, 후추, 소금, 파슬리가루, 슬라이스치즈 2장.
  
  2. **파스타 삶기**: 끓는 물에 소금을 넣고 파스타면을 넣어 포장지에 적힌 시간에 맞춰 삶습니다. 삶은 후 체에 걸러 물기를 빼고, 올리브유를 약간 넣어 섞어줍니다.

  3. **삼겹살 볶기**: 팬에 올리브유를 두르고 중불에서 다진마늘을 넣어 향이 올라오도록 볶습니다. 그 후 삼겹살을 추가하고, 겉이 바삭하게 익을 때까지 볶습니다.

  4. **재료 혼합**: 삼겹살이 익으면 청양고추와 버섯플레이크를 넣고, 후추와 소금으로 간을 맞춥니다. 잘 섞어 2-3분 더 볶아줍니다.

  5. **파스타와 혼합**: 삶은 파스타를 팬에 넣고 모든 재료가 잘 섞이도록 볶습니다. 원한다면 추가로 소금을 더할 수 있습니다.

  6. **마무리**: 불을 끄고 슬라이스치즈를 위에 올려 녹을 때까지 뚜껑을 덮어둡니다. 마지막으로 파슬리가루를 뿌려 장식합니다.

  7. **서빙**: 접시에 담아 따뜻하게 서빙합니다. 매콤하고 고소한 삼겹살 파스타를 즐기세요!

### [실습] 템플릿을 변경하여 나만의 헬스 트레이너 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 헬스 트레이너
2. 운동하고 싶은 신체 부위를 입력하면 운동 루틴을 출력 
3. `{today}` 에는 운동하고 싶은 신체 부위를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [ ]:
# 코드 작성